# 1. 参数的更新

神经网络的学习的目的是找到使损失函数的值尽可能小的参数。这是寻找最优参数的问题，解决这个问题的过程称为最优化（optimization）。遗憾的是，神经网络的最优化问题非常难。这是因为参数空间非常复杂，无法轻易找到最优解。而且，在深度神经网络中，参数的数量非常庞大，导致最优化问题更加复杂。

## 1.1 SGD

随机梯度下降法（Stochastic Gradient Descent，SGD）是一般机器学习中应用最多的优化算法，特别是在深度学习中。按照数据生成分布抽取 m 个小批量（独立同分布的）样本，通过计算它们的梯度均值，我们可以得到梯度的无偏估计。SGD 是一个简单的方法，不过比起胡乱地搜索参数空间，也算是“聪明”的方法。

$$ W = W - \eta \frac{\partial L}{\partial W} $$

这里把需要更新的权重参数记为 $W$，把损失函数关于 $W$ 的梯度记为 $\frac{\partial L}{\partial W}$。$\eta$ 表示学习率（learning rate），之前实践中，所使用的都是固定的学习率。若学习率太大，学习曲线会剧烈震荡；相反，如果学习率太小，学习过程会很缓慢，可能会卡在一个相当高的代价值。在实践中，有必要随着时间的推移逐渐降低学习率，一般会线性衰减学习率直到第n次迭代：

$$ \eta_k = (1 - \alpha)\eta_0 + \alpha \eta_n $$

其中 $ \alpha = \frac{k}{n}$。通常 $ \eta_n $ 应设为大约 $\eta_0$的 1%。

现在，我们将 SGD 算法实现为一个名为 SGD 的 Python 类，

In [2]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

进行初始化时的参数 lr 表示学习率。代码段中还定义了update(params, grads)方法，这个方法在SGD中会被反复调用。参数 params 和 grads（与之前的神经网络的实现一样）是字典型变量，按 params['W1']、grads['W1'] 的形式，分别保存了权重参数和它们的梯度。

可以按如下方式进行神经网络的参数的更新：

```python
network = TwoLayerNet(...)
optimizer = SGD()    # 优化器对象

for i in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...) # mini-batch
    grads = network.gradient(x_batch, t_batch)
    params = network.params
    optimizer.update(params, grads)
    ...
```

虽然SGD简单容易实现，但是在解决某些问题时可能没有效率。这里思考一下求下面这个函数的最小值的问题。

$$ f(x, y) = \frac{1}{20}x^2 + y^2 $$

这个函数梯度的特征是，y 轴方向上大，x 轴方向上小。我们尝试对该函数应用 SGD，从 $(x,y)=(-7, 2)$ 初始点开始搜索，如下图所示，搜索路径呈“之”字形移动。

![img](images/chapter12/SGD.png)

SGD低效的根本原因是，梯度的方向并没有指向最小值的方向。为了改正 SGD 的缺点，下面我们将介绍 Momentum、AdaGrad、Adam 这三种方法来取代SGD。

## 1.2 动量(Momentum)

虽然 SGD 仍然是非常受欢迎的优化方法，但其学习过程有时会很慢。动量方法旨在加速学习，特别是处理小但一致的梯度，或是带噪声的梯度。动量算法积累了之前梯度指数级衰减的移动平均，并且继续沿该方向移动。

数学表示动量方法如下：
$$ v = \alpha v - \eta \frac{\partial L}{\partial W} $$
$$ W = W + v $$

之前步长只是梯度乘以学习率，现在步长取决于梯度序列的大小和排列。当许多连续的梯度指向相同的方向时，步长最大。如果动量算法总是观测到梯度 g，那么它会不停加速，直到达到最终速度，其中步长大小为

$$ v_{(n)} =  \alpha^n v_{(0)} -\frac{1-\alpha^n}{1-\alpha}\cdot \eta \frac{\partial L}{\partial W} \approx -\frac{\eta}{1-\alpha}\cdot \frac{\partial L}{\partial W} $$

因此将动量的超参数视为 $\frac{1}{1-\alpha}$ 有助于理解。例如，$\alpha = 0.9$ 对应着最大速度10倍于梯度下降算法。和学习率一样，$\alpha$ 也会随着时间不段调整，一般初始值是一个较小的值，随后慢慢变大。

In [3]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

和 SGD 的情形相比，动量方法可以更快地朝 x 轴方向靠近，减弱“之”字形的变动程度。这是因为虽然 x 轴方向上受到的力非常小，但是一直在同一方向上受力，所以朝同一个方向会有一定的加速。相反，虽然 y 轴方向上受到的力很大，但是因为交互地受到正方向和反方向的力，它们会互相抵消。

![img](images/chapter12/momentum.png)

## 1.3 AdaGrad